1) Reading the CSV Files

In [66]:
import csv
from collections import defaultdict


def read_texts_from_csv(path, text_col_name):
    texts = []
    with open(path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            texts.append(row[text_col_name])
    return texts

train_texts = read_texts_from_csv('train.csv', 'Text')
val_texts   = read_texts_from_csv('val.csv',   'Text')
test_texts  = read_texts_from_csv('test.csv',  'Text')
sample_texts= read_texts_from_csv('sample.csv','Truncated Text')



2) Tokenization and Preprocessing

In [67]:
def simple_tokenize(sentence):
    return sentence.strip().split()

def add_start_end_tokens(token_lists):
    out = []
    for tok_list in token_lists:
        out.append(['<s>'] + tok_list + ['</s>'])
    return out

# Convert train_texts -> train_raw_tokens
train_raw_tokens = []
for txt in train_texts:
    raw = simple_tokenize(txt)
    train_raw_tokens.append(raw)
train_raw_tokens = add_start_end_tokens(train_raw_tokens)


Building a Vocabulary that includes <unk>

In [68]:
def build_vocabulary(train_tokens, min_count=2):
    freq = defaultdict(int)
    for sent in train_tokens:
        for w in sent:
            freq[w] += 1

    vocab = ['<unk>']
    for w, c in freq.items():
        if c >= min_count:
            vocab.append(w)
    vocab_dict = {w: i for i, w in enumerate(sorted(vocab))}
    return vocab_dict

vocab_dict = build_vocabulary(train_raw_tokens, min_count=2)
print("Vocab size =", len(vocab_dict))

# Step 4: replace OOV with <unk> in train
def replace_oov_with_unk(token_lists, vocab_dict):
    new_lists = []
    for tokens in token_lists:
        new_tokens = []
        for t in tokens:
            if t not in vocab_dict:
                new_tokens.append('<unk>')
            else:
                new_tokens.append(t)
        new_lists.append(new_tokens)
    return new_lists

train_tokens = replace_oov_with_unk(train_raw_tokens, vocab_dict)

# For validation
val_raw_tokens = []
for txt in val_texts:
    val_raw_tokens.append(simple_tokenize(txt))
val_raw_tokens = add_start_end_tokens(val_raw_tokens)
val_tokens = replace_oov_with_unk(val_raw_tokens, vocab_dict)

V = len(vocab_dict)


# For test
test_raw_tokens = []
for txt in test_texts:
    raw = simple_tokenize(txt)
    test_raw_tokens.append(raw)

test_raw_tokens = add_start_end_tokens(test_raw_tokens)
test_tokens = replace_oov_with_unk(test_raw_tokens, vocab_dict)




Vocab size = 34388


3) Part 1: N-gram Language Models

-   `unigram_counts[(w,)]` is the count of word `w` in the entire training data.

-   `bigram_counts[(w1,w2)]` is the count of the pair `(w1, w2)`.

-   `trigram_counts[(w1,w2,w3)]` is the count of the triple `(w1, w2, w3)`.     


p(w2|w1) = count(w1,w2)/count(w1) and  . . .   

In [69]:
from collections import defaultdict

# Step 5: re-count n-grams with the final (unk-ified) train tokens
def count_ngrams(token_lists, n):
    ngram_counts = defaultdict(int)
    for tokens in token_lists:
        if len(tokens) < n: 
            continue
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngram_counts[ngram] += 1
    return ngram_counts

unigram_counts = count_ngrams(train_tokens, 1)
bigram_counts  = count_ngrams(train_tokens, 2)
trigram_counts = count_ngrams(train_tokens, 3)

zerogram_total = sum(unigram_counts.values())




### 3.2 Smoothing


-   **Dirichlet smoothing** for the unigram model (equivalent to additive smoothing):

    ![Dirichlet smoothing](./images/Dirichlet%20smoothing.png)


    where `N` is the total number of words in the training data, `V` is the size of the vocabulary, and `α` is a hyperparameter that can be tuned on a validation set.


-   **Kneser-Ney smoothing** for the bigram and trigram cases. A simplified Kneser-Ney for bigrams:

    ![Kneser-Ney smoothing](./images/Kneser-Ney%20smoothing.png)



For **trigram** Kneser-Ney, a similar hierarchical formula:

![trigram smoothing](./images/trigram%20Kneser-Ney.png)


**Note**: The discount `d` (sometimes `D`) is often learned or tuned on validation data. Typically it's in `(0,1)`.

In [70]:
from collections import defaultdict

class NGramModel:
    def __init__(self, n, vocab_dict, ngram_counts, lower_order_counts=None, total_unigrams=None, discount=0.75, alpha=0.1):
        """
        n = 1,2,3
        vocab_dict: dictionary mapping tokens -> ids (optional usage)
        ngram_counts: dict for n-gram counts
        lower_order_counts: dict for (n-1)-gram counts (for n>=2)
        total_unigrams: total count of all unigrams (for unigrams normalizing)
        discount: for Kneser-Ney
        alpha: for Dirichlet smoothing in the unigram case
        """
        self.n = n
        self.vocab_dict = vocab_dict
        self.ngram_counts = ngram_counts
        self.lower_order_counts = lower_order_counts
        self.total_unigrams = total_unigrams
        self.discount = discount
        self.alpha = alpha
        

        if n == 2:
            # how many distinct contexts lead to each word w2
            self.num_preceding_contexts = defaultdict(int)
            for (w1, w2), c in ngram_counts.items():
                self.num_preceding_contexts[w2] += 1
            self.total_bigram_types = len(ngram_counts)
            
            self.num_bigrams_starting_with = defaultdict(int)
            for (w1, w2), count in ngram_counts.items():
                self.num_bigrams_starting_with[w1] += 1


        elif n == 3:
            # We still need the distinct followers data for the trigram discount:
            self.num_distinct_following = defaultdict(set)
            for (w1, w2, w3), c in ngram_counts.items():
                self.num_distinct_following[(w1, w2)].add(w3)
            self.total_trigram_types = len(ngram_counts)


            self.num_preceding_contexts = defaultdict(int)
            for (x1, x2), c in lower_order_counts.items():
                self.num_preceding_contexts[x2] += 1
            self.total_bigram_types = len(lower_order_counts)

            self.num_bigrams_starting_with = defaultdict(int)
            for (x1, x2), c in lower_order_counts.items():
                self.num_bigrams_starting_with[x1] += 1


    # Dirichlet (add-alpha) smoothing
    def unigram_prob(self, w):
        count_w = self.ngram_counts.get((w,), 0)
        numerator = count_w + self.alpha
        denominator = self.total_unigrams + self.alpha * len(self.vocab_dict)
        return numerator / denominator


    # Kneser-Ney for bigrams
    def bigram_prob_kn(self, w1, w2):
        # P(w2|w1) = max(count(...) - d, 0) / count(w1) + lambda(...) * P_continuation(w2)
        
        bigram_count = self.ngram_counts.get((w1, w2), 0)
        unigram_count = self.lower_order_counts.get((w1,), 0) 
        
        main_term = max(bigram_count - self.discount, 0.0) / (unigram_count if unigram_count > 0 else 1)
        
        lambda_w1 = (self.discount * self.num_bigrams_starting_with[w1]) / (unigram_count if unigram_count > 0 else 1)
        
        cont_prob = self.num_preceding_contexts[w2] / self.total_bigram_types
        
        return main_term + lambda_w1 * cont_prob


    # Kneser-Ney for trigram 
    def trigram_prob_kn(self, w1, w2, w3):
        # P(w3|w1,w2) = max(count(...) - d, 0) / count(w1,w2) + lambda(...) * bigram_prob_kn(w2,w3)
        
        tri_count  = self.ngram_counts.get((w1, w2, w3), 0)
        bi_count   = self.lower_order_counts.get((w1, w2), 0)
        main_term  = max(tri_count - self.discount, 0.0) / (bi_count if bi_count > 0 else 1)
        
        num_distinct = len(self.num_distinct_following[(w1, w2)])
        lambda_w1_w2 = (self.discount * num_distinct / (bi_count if bi_count > 0 else 1))
        
        return main_term + lambda_w1_w2 * self.bigram_prob_kn(w2, w3)


    def ngram_prob(self, context, w):
        """
        context: tuple of length n-1
        w: the next word
        Return the smoothed probability of word w given context (depending on self.n).
        """
        if self.n == 1:
            return self.unigram_prob(w)
        elif self.n == 2:
            w1 = context[-1]
            return self.bigram_prob_kn(w1, w)
        else:
            w1, w2 = context[-2], context[-1]
            return self.trigram_prob_kn(w1, w2, w)


### 3.3 Choosing/Optimizing Hyperparameters on Validation


-   `alpha` for your **unigram** Dirichlet smoothing,

-   `discount` for your Kneser-Ney in bigram/trigram.   

-   `perplexity` is a common metric in language modeling that indicates how well a model predicts a sample of text. A lower perplexity means the model is more "certain" (or places higher probability) on the actual sequence of words.

In [71]:
import math


def perplexity(model, token_lists):
    """
    Computes perplexity for any N-gram model (unigram, bigram, trigram).
    """
    log_prob_sum = 0.0
    total_count  = 0

    n = model.n
    for tokens in token_lists:
        if len(tokens) < n:
            continue

        for i in range(n-1, len(tokens)):
            context = tuple(tokens[i - (n - 1) : i])
            w = tokens[i]
            p = model.ngram_prob(context, w)
            log_prob_sum += math.log(max(p, 1e-15))
            total_count += 1

    if total_count == 0:
        return float('inf')

    return math.exp(-log_prob_sum / total_count)



def build_ngram_model(
    n,
    vocab_dict,
    unigram_counts,
    bigram_counts,
    trigram_counts,
    total_unigrams,
    alpha=0.1,
    discount=0.75,
):
    """
    Creates and returns an NGramModel of order n with the specified
    hyperparameters alpha (for unigram Dirichlet) and discount (for
    Kneser-Ney in bigrams/trigrams).
    """
    if n == 1:
        # For unigrams, alpha smoothing
        return NGramModel(
            n=1,
            vocab_dict=vocab_dict,
            ngram_counts=unigram_counts,
            total_unigrams=total_unigrams,
            alpha=alpha,
        )
    elif n == 2:
        # For bigrams, discount for Kneser-Ney
        return NGramModel(
            n=2,
            vocab_dict=vocab_dict,
            ngram_counts=bigram_counts,
            lower_order_counts=unigram_counts,
            total_unigrams=total_unigrams,
            discount=discount,
        )
    else:
        # trigram
        return NGramModel(
            n=3,
            vocab_dict=vocab_dict,
            ngram_counts=trigram_counts,
            lower_order_counts=bigram_counts,
            total_unigrams=total_unigrams,
            discount=discount,
        )




# grid search over possible α and discount values
possible_alphas    = [0.01, 0.1, 0.5, 0.9]
possible_discounts = [0.1, 0.5, 0.6, 0.75]


######################
# UNIGRAM 
######################

best_alpha_uni = None
best_val_pp_uni = float('inf')

for alpha in possible_alphas:
    model_uni = build_ngram_model(
        n=1,
        vocab_dict=vocab_dict,
        unigram_counts=unigram_counts,
        bigram_counts=bigram_counts,
        trigram_counts=trigram_counts,
        total_unigrams=zerogram_total,
        alpha=alpha
        )
    pp_val = perplexity(model_uni, val_tokens)
    if pp_val < best_val_pp_uni:
        best_val_pp_uni = pp_val
        best_alpha_uni  = alpha

print(f"[Unigram] best alpha = {best_alpha_uni}, val perplexity = {best_val_pp_uni}")


######################
# BIGRAM 
######################

best_discount_bi = None
best_val_pp_bi   = float('inf')

for d in possible_discounts:
    model_bi = build_ngram_model(
        n=2,
        vocab_dict=vocab_dict,
        unigram_counts=unigram_counts,
        bigram_counts=bigram_counts,
        trigram_counts=trigram_counts,
        total_unigrams=zerogram_total,
        discount=d
    )
    pp_val = perplexity(model_bi, val_tokens)
    if pp_val < best_val_pp_bi:
        best_val_pp_bi = pp_val
        best_discount_bi = d

print(f"[Bigram] best discount = {best_discount_bi}, val perplexity = {best_val_pp_bi}")


######################
# TRIGRAM 
######################

best_discount_tri = None
best_val_pp_tri   = float('inf')

for d in possible_discounts:
    model_tri = build_ngram_model(
        n=3,
        vocab_dict=vocab_dict,
        unigram_counts=unigram_counts,
        bigram_counts=bigram_counts,
        trigram_counts=trigram_counts,
        total_unigrams=zerogram_total,
        discount=d
    )
    pp_val = perplexity(model_tri, val_tokens)
    if pp_val < best_val_pp_tri:
        best_val_pp_tri = pp_val
        best_discount_tri = d

print(f"[Trigram] best discount = {best_discount_tri}, val perplexity = {best_val_pp_tri}")
print()

ppl_train_uni = perplexity(model_uni, train_tokens)
ppl_train_bi  = perplexity(model_bi,  train_tokens)
ppl_train_tri = perplexity(model_tri, train_tokens)
print("Train perplexities:", ppl_train_uni, ppl_train_bi, ppl_train_tri)






[Unigram] best alpha = 0.01, val perplexity = 1308.1129901488932
[Bigram] best discount = 0.75, val perplexity = 204.19341350665897
[Trigram] best discount = 0.75, val perplexity = 6082.749435511439

Train perplexities: 1651.5449061453303 73.5026099910924 3.9270501360394254


### 3.4 Evaluate on Test Set

After determining the best hyperparameters, we finalize our model then measure perplexity on `test`.

In [72]:
# Rebuild best models using chosen hyperparams

best_uni_model = build_ngram_model(
    n=1,
    vocab_dict=vocab_dict,
    unigram_counts=unigram_counts,
    bigram_counts=bigram_counts,
    trigram_counts=trigram_counts,
    total_unigrams=zerogram_total,
    alpha=best_alpha_uni
)

best_bi_model = build_ngram_model(
    n=2,
    vocab_dict=vocab_dict,
    unigram_counts=unigram_counts,
    bigram_counts=bigram_counts,
    trigram_counts=trigram_counts,
    total_unigrams=zerogram_total,
    discount=best_discount_bi 
)

best_tri_model = build_ngram_model(
    n=3,
    vocab_dict=vocab_dict,
    unigram_counts=unigram_counts,
    bigram_counts=bigram_counts,
    trigram_counts=trigram_counts,
    total_unigrams=zerogram_total,
    discount=best_discount_tri
)

# compute perplexities on the test set:
test_pp_uni = perplexity(best_uni_model, test_tokens)
test_pp_bi  = perplexity(best_bi_model,  test_tokens)
test_pp_tri = perplexity(best_tri_model, test_tokens)

print("\n=== TEST PERPLEXITIES ===")
print(f"Unigram model perplexity: {test_pp_uni}")
print(f"Bigram  model perplexity: {test_pp_bi}")
print(f"Trigram model perplexity: {test_pp_tri}")
print()

lowest_pp = min(test_pp_uni, test_pp_bi, test_pp_tri)
if lowest_pp == test_pp_uni:
    print("Unigram is best on test set.")
elif lowest_pp == test_pp_bi:
    print("Bigram is best on test set.")
else:
    print("Trigram is best on test set.")


=== TEST PERPLEXITIES ===
Unigram model perplexity: 1312.1047027555612
Bigram  model perplexity: 204.06683231216311
Trigram model perplexity: 5738.741642152637

Bigram is best on test set.


### 4) Part 2: Feed-Forward Neural Network
--------------------------------------

Here, you must:

1.  Represent the input context (unigram, bigram, trigram) in some way. Two common ways:

    -   **One-hot** of the context words. (If `n=3` (trigram), you have 2 words in context, each one-hot if the vocabulary is not too large.)

    -   **Train an embedding** from scratch: each word has a small embedding vector that you learn simultaneously (similar to word2vec).

2.  Build a basic feed-forward network with at least one hidden layer.

3.  Train it to **predict the next word** from the context.

4.  Tune learning rate (and possibly hidden-layer size) using the validation set.

5.  Evaluate perplexity on the test set, same formula:

    ![perplexity](./images/perplexity.png)

    where `N` is the total number of words in the test set and `p(w_i|w_{i-1},w_{i-2})` is the probability assigned by your model.

> **Note**: Usually, you'd use PyTorch/TensorFlow to handle all details. Since **libraries are not allowed** for the fundamental logic, you must implement your own forward/backprop. **(You may or may not be allowed to use `numpy` for basic matrix multiplies.)**

In [ ]:
!pip install tqdm

In [73]:
import math
import random
from tqdm import tqdm


##############################################
# Feed-Forward Network
##############################################

class SimpleFeedForwardNN:
    def __init__(
        self,
        vocab_size,
        embed_dim=50,
        hidden_size=100,
        context_size=2,
        learning_rate=0.01,
    ):
        """
        vocab_size: total number of words in the vocabulary
        embed_dim: dimension of embeddings for each word
        hidden_size: number of hidden units
        context_size: how many words form the context (2 for trigram)
        """
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_size = hidden_size
        self.context_size = context_size
        self.lr = learning_rate

        # Each row corresponds to embedding of one word in the vocabulary [vocab_size, embed_dim]
        self.embeddings = [
            [(random.random() - 0.5) * 0.01 for _ in range(embed_dim)]
            for _ in range(vocab_size)
        ]

        # input to the hidden layer is a concatenation of all context word embeddings
        self.W1 = [
            [(random.random() - 0.5) * 0.01 for _ in range(hidden_size)]
            for _ in range(context_size * embed_dim)
        ]
        
        # Output layer 
        # Each row corresponds to one hidden neuron, and each column corresponds to an output dimension
        self.W2 = [
            [(random.random() - 0.5) * 0.01 for _ in range(vocab_size)]
            for _ in range(hidden_size)
        ]



    def forward(self, context_word_ids):
        
        # 1) embed each context word, then concatenate
        x = []
        for wid in context_word_ids:
            x.extend(self.embeddings[wid])

        # 2) Hidden layer computation (input @ hidden) 
        hidden = [0] * self.hidden_size
        for j in range(self.hidden_size):
            s = 0
            for i in range(len(x)):
                s += x[i] * self.W1[i][j]
            hidden[j] = math.tanh(s)

        # 3) output logits =  (hidden @ output)
        # unnormalized scores for each word in the vocabulary
        logits = [0] * self.vocab_size
        for k in range(self.vocab_size):
            s = 0
            for j in range(self.hidden_size):
                s += hidden[j] * self.W2[j][k]
            logits[k] = s

        # 4) softmax
        max_logit = max(logits)
        exps = [math.exp(log - max_logit) for log in logits]
        sum_exps = sum(exps)
        probs = [e / sum_exps for e in exps]

        # input vector x, the hidden layer output and (model’s prediction distribution over the vocabulary) 
        return x, hidden, probs


    def backward(self, x, hidden, probs, target_word_id, context_word_ids):
        # 1) derivative wrt logits = (pred - actual)
        dlogits = [p for p in probs]
        dlogits[target_word_id] -= 1.0

        # 2) update W2 using dlogits * hidden
        dW2 = [[0]*self.vocab_size for _ in range(self.hidden_size)]
        for j in range(self.hidden_size):
            for k in range(self.vocab_size):
                dW2[j][k] = hidden[j] * dlogits[k]

        # 3) derivative wrt hidden
        dhidden = [0] * self.hidden_size
        for j in range(self.hidden_size):
            s = 0
            for k in range(self.vocab_size):
                s += dlogits[k] * self.W2[j][k]
            dhidden[j] = s*(1 - hidden[j] * hidden[j])  # derivative of tanh

        # 4) update W1
        dW1 = [[0]*self.hidden_size for _ in range(len(x))]
        for i in range(len(x)):
            for j in range(self.hidden_size):
                dW1[i][j] = x[i]*dhidden[j]

        # 5) apply gradient updates to W2
        for j in range(self.hidden_size):
            for k in range(self.vocab_size):
                self.W2[j][k] -= self.lr*dW2[j][k]

        # 6) apply gradient updates to W1
        for i in range(len(x)):
            for j in range(self.hidden_size):
                self.W1[i][j] -= self.lr*dW1[i][j]

        # 7) derivative wrt embeddings
        dembeddings = [[0]*self.embed_dim for _ in range(self.context_size)]
        for c_i in range(self.context_size):
            for j in range(self.hidden_size):
                for e_i in range(self.embed_dim):
                    i_global = c_i*self.embed_dim + e_i
                    dembeddings[c_i][e_i] += dhidden[j]*self.W1[i_global][j]

        # update embeddings
        for c_i, wid in enumerate(context_word_ids):
            for e_i in range(self.embed_dim):
                self.embeddings[wid][e_i] -= self.lr*dembeddings[c_i][e_i]

    def train_on_example(self, context_word_ids, target_word_id):
        x, hidden, probs = self.forward(context_word_ids)
        self.backward(x, hidden, probs, target_word_id, context_word_ids)

    def predict_next_word_prob(self, context_word_ids):
        _, _, probs = self.forward(context_word_ids)
        return probs


### evaluation and paramter tuning



In [ ]:
from tqdm import tqdm

##########################################################
# Convert sequences of tokens into pairs (context, target)
##########################################################

def build_context_target_pairs(token_lists, n, vocab_dict):
    """
    For n=1 (unigram model), context_size=0 => we can feed an empty context or
    a special <NULL> token to the network.

    For n=2 (bigram), context_size=1 => each training example is (w_{i-1}, w_i).
    For n=3 (trigram), context_size=2 => (w_{i-2}, w_{i-1}, w_i).
    etc.
    """
    pairs = []
    context_size = n - 1

    for tokens in tqdm(token_lists, desc="Building context-target pairs", unit="sentence"):
        if len(tokens) < n:
            continue
        for i in range(context_size, len(tokens)):
            context_tokens = tokens[i - context_size : i]
            target_token = tokens[i]
            context_ids = [vocab_dict.get(t, 0) for t in context_tokens]
            target_id = vocab_dict.get(target_token, 0)
            pairs.append((context_ids, target_id))
            
    return pairs


##########################################
# 3) Perplexity for the feed-forward model
##########################################

def perplexity_ffnn(nn_model, pairs):
    """
    Calculate perplexity from (context, target) pairs directly.
    sum -log p(target|context) across all pairs, then exponentiate.
    """
    log_prob_sum = 0.0
    total_count = 0

    for (ctx_ids, tgt_id) in tqdm(pairs, desc="Calculating perplexity", unit="pair"):
        probs = nn_model.predict_next_word_prob(ctx_ids)
        p = probs[tgt_id]
        if p <= 0:
            log_prob_sum += -999999
        else:
            log_prob_sum += math.log(p)
        total_count += 1
        

    if total_count == 0:
        return float
    ("inf")

    return math.exp(-log_prob_sum / total_count)


############################################
# tries different learning rates, picks best 
############################################

def train_ffnn_for_ngram(
    n,
    train_tokens,
    val_tokens,
    vocab_dict,
    embed_dim=50,
    hidden_size=100,
    learning_rates=[0.01, 0.1, 0.5, 1.0],
    epochs=5,
):
    """
    For a given n (1,2,3):
    1) Build (context, target) pairs for train, val
    2) Loop over possible learning_rates
    3) For each LR, train a new model for epochs
    4) Track the best validation perplexity
    5) Return the best model (or its LR)
    """

    train_pairs = build_context_target_pairs(train_tokens, n, vocab_dict)
    val_pairs = build_context_target_pairs(val_tokens, n, vocab_dict)

    best_lr = None
    best_val_pp = float("inf")
    best_model = None

    for lr in learning_rates:
        context_size = max(n - 1, 1)
        nn = SimpleFeedForwardNN(
            vocab_size=len(vocab_dict),
            embed_dim=embed_dim,
            hidden_size=hidden_size,
            context_size=context_size,
            learning_rate=lr,
        )

        for ep in range(epochs):
            random.shuffle(train_pairs)
            for (ctx, tgt) in tqdm(train_pairs, desc=f"Epoch {ep+1}/{epochs}", unit="batch"):
                nn.train_on_example(ctx, tgt)

            val_pp = perplexity_ffnn(nn, val_pairs)

        final_val_pp = perplexity_ffnn(nn, val_pairs)
        if final_val_pp < best_val_pp:
            best_val_pp = final_val_pp
            best_lr = lr
            best_model = nn


    print(f"** For n={n}, best LR={best_lr}, perplexity={best_val_pp}")
    
    return best_model  


### 5.1 Best Model from Part 1 or Part 2


In [ ]:
# Reduce the training and validation data size for faster debugging
# train_tokens_sample = train_tokens[:int(0.1 * len(train_tokens))]  # Use 10% of the data
# val_tokens_sample = val_tokens[:int(0.1 * len(val_tokens))]

train_tokens_sample = train_tokens[:int(1 * len(train_tokens))]
val_tokens_sample = val_tokens[:int(1 * len(val_tokens))]

    
# (1) Train a unigram feed-forward model
ffnn_unigram = train_ffnn_for_ngram(
    n=1,
    train_tokens=train_tokens_sample,
    val_tokens=val_tokens_sample,
    vocab_dict=vocab_dict,
    embed_dim=50,
    hidden_size=100,
    learning_rates=[0.001, 0.01, 0.1, 0.5],
    epochs=2
)

# (2) Train a BIGRAM feed-forward model
ffnn_bigram = train_ffnn_for_ngram(
    n=2,
    train_tokens=train_tokens_sample,
    val_tokens=val_tokens_sample,
    vocab_dict=vocab_dict,
    embed_dim=50,
    hidden_size=100,
    learning_rates=[0.001, 0.01, 0.1, 0.5],
    epochs=2
)

# (3) Train a TRIGRAM feed-forward model
ffnn_trigram = train_ffnn_for_ngram(
    n=3,
    train_tokens=train_tokens_sample,
    val_tokens=val_tokens_sample,
    vocab_dict=vocab_dict,
    embed_dim=50,
    hidden_size=100,
    learning_rates=[0.001, 0.01, 0.1, 0.5],
    epochs=2
)

# Evaluate all on TEST
test_pairs_unigram = build_context_target_pairs(test_tokens, 1, vocab_dict)
test_pairs_bigram  = build_context_target_pairs(test_tokens, 2, vocab_dict)
test_pairs_trigram = build_context_target_pairs(test_tokens, 3, vocab_dict)

test_pp_uni_nn = perplexity_ffnn(ffnn_unigram,  test_pairs_unigram)
test_pp_bi_nn  = perplexity_ffnn(ffnn_bigram,   test_pairs_bigram)
test_pp_tri_nn = perplexity_ffnn(ffnn_trigram,  test_pairs_trigram)

print("\n=== TEST PERPLEXITIES (Feed-Forward) ===")
print(f"Unigram   => {test_pp_uni_nn}")
print(f"Bigram    => {test_pp_bi_nn}")
print(f"Trigram   => {test_pp_tri_nn}")
print()

test_pp_uni_nn = perplexity(best_uni_model, test_tokens)
test_pp_bi_nn  = perplexity(best_bi_model,  test_tokens)
test_pp_tri_nn = perplexity(best_tri_model, test_tokens)



print("\nNow comparing with best n-gram results from Section 1:")
print(f"Unigram n-gram perplexity:  {test_pp_uni}")
print(f"Bigram n-gram perplexity:   {test_pp_bi}")
print(f"Trigram n-gram perplexity:  {test_pp_tri}")
print()
print("\nFeed-Forward Neural Network:")
print(f"Unigram n-gram perplexity:  {test_pp_uni_nn}")
print(f"Bigram n-gram perplexity:   {test_pp_bi_nn}")
print(f"Trigram n-gram perplexity:  {test_pp_tri_nn}")


Calculating perplexity: 100%|██████████| 13136/13136 [04:02<00:00, 54.09pair/s]


** For n=1, best LR=0.01, perplexity=34388.00000004098


Calculating perplexity: 100%|██████████| 13030/13030 [04:59<00:00, 43.56pair/s]


** For n=2, best LR=0.01, perplexity=34387.999931053615


Calculating perplexity: 100%|██████████| 12924/12924 [05:01<00:00, 42.86pair/s]


** For n=3, best LR=0.1, perplexity=34387.99996559879


Calculating perplexity: 100%|██████████| 139390/139390 [49:08<00:00, 47.27pair/s]



=== TEST PERPLEXITIES (Feed-Forward) ===
Unigram   => 34388.00000038776
Bigram    => 34387.99991768191
Trigram   => 34387.999920531416


Now comparing with best n-gram results from Section 1:
Unigram n-gram perplexity:  1312.1047027555612
Bigram n-gram perplexity:   204.06683231216311
Trigram n-gram perplexity:  5738.741642152637


Feed-Forward Neural Network:
Unigram n-gram perplexity:  1312.1047027555612
Bigram n-gram perplexity:   204.06683231216311
Trigram n-gram perplexity:  5738.741642152637




### 5.1 Best Model from Part 1 or Part 2



In [76]:

model_perplexities = {
    "Unigram N-gram":  (test_pp_uni,     best_uni_model),
    "Bigram N-gram":   (test_pp_bi,      best_bi_model),
    "Trigram N-gram":  (test_pp_tri,     best_tri_model),
    "Unigram NN":      (test_pp_uni_nn,  ffnn_unigram),
    "Bigram NN":       (test_pp_bi_nn,   ffnn_bigram),
    "Trigram NN":      (test_pp_tri_nn,  ffnn_trigram),
}

best_model_name = None
best_pp = float('inf')
best_model_obj = None

for name, (pp, model_obj) in model_perplexities.items():
    if pp < best_pp:
        best_pp = pp
        best_model_name = name
        best_model_obj  = model_obj

print(f"\n=> The best overall model is: {best_model_name} with perplexity = {best_pp}")



=> The best overall model is: Bigram N-gram with perplexity = 204.06683231216311


In [77]:
import csv
from tqdm import tqdm

def predict_next_word_ngram(model, tokens):
    """
    Predict SINGLE next word given the last (n-1) tokens
    Returns the predicted word
    """
    n = model.n
    context_size = n - 1

    if len(tokens) < context_size:
        context = tuple(["<s>"] * (context_size - len(tokens)) + tokens)
    else:
        context = tuple(tokens[-context_size:])

    best_word = None
    best_prob = -1

    for word in model.vocab_dict.keys():
        p = model.ngram_prob(context, word)
        if p > best_prob:
            best_prob = p
            best_word = word
            
            
    return best_word


def predict_multiple_words_ngram(model, tokens, max_words=4):
    """
    predict up to 'max_words' missing words to reach end of sentence.
    For each next word, we call 'predict_next_word_ngram', then append that word.
    Stop if we generate '</s>'
    """
    predicted_words = []
    for _ in tqdm(range(max_words), desc="Predicting words", unit="word"):
        next_w = predict_next_word_ngram(model, tokens)
        predicted_words.append(next_w)
        tokens.append(next_w)
        
        if next_w == "</s>":
            break
        
    return predicted_words


def read_rows_from_csv(path):
    """
    Read an entire CSV into a list of dictionaries.
    """
    with open(path, mode="r", encoding="utf-8") as fin:
        reader = csv.DictReader(fin)
        rows = list(reader)
    return rows


def generate_csv_predictions_ngram(
    input_csv="sample.csv",
    output_csv="sample_output.csv",
    text_col_name="Truncated text",
    best_model=None,
):
    """
    Read sample.csv, use best_model to predict:
      (1) first missing word,
      (2) up to 4 words,
    then write to sample_output.csv with new columns.
    """
    with open(input_csv, mode="r", encoding="utf-8") as fin, open(
        output_csv, mode="w", encoding="utf-8", newline=""
    ) as fout:
        reader = csv.DictReader(fin)
        fieldnames = reader.fieldnames + [
            "FirstPredictedWord",
            "FullPrediction",
        ]
        writer = csv.DictWriter(fout, fieldnames=fieldnames)
        writer.writeheader()

        for row in tqdm(reader, desc="Processing rows", unit="row"):
            truncated_text = row[text_col_name].strip()
            tokens = truncated_text.split()

            # 1) Predict the first missing word
            first_word = predict_next_word_ngram(best_model, tokens)

            # 2) predict up to 4 missing words
            predicted_sequence = predict_multiple_words_ngram(
                best_model, tokens[:], max_words=4
            )

            row["FirstPredictedWord"] = first_word
            row["FullPrediction"] = " ".join(predicted_sequence)
            writer.writerow(row)

    print(f"Predictions written to {output_csv}.")



sample_rows = read_rows_from_csv("sample.csv")

generate_csv_predictions_ngram(
    input_csv="sample.csv",
    output_csv="sample_output.csv",
    text_col_name="Truncated Text",
    best_model=best_model_obj
)


Predicting words: 100%|██████████| 4/4 [00:00<00:00, 19.42word/s]
Processing rows: 99row [00:25,  3.93row/s]

Predictions written to sample_output.csv.
